# --- Day 15: Beverage Bandits ---

In [3]:
t1 = '''#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######'''

In [4]:
from collections import defaultdict

In [82]:
class Grid():
    
    def __init__(self, initialmap):
        """
        """
        self.opencells = []
        self.g_loc = {}
        self.e_loc = {} 
        g=0; e=0
        for y, row in enumerate(initialmap):
            for x, c in enumerate(row):
                if c != '#':
                    self.opencells.append((y,x)) 
                else:
                    pass
                if c == 'G':
                    g += 1
                    self.g_loc['G{}'.format(g)] = (y,x)
                elif c == 'E':
                    e += 1
                    self.e_loc['E{}'.format(e)] = (y,x)
                else:
                    pass

        self.g_points = {}
        self.e_points = {}
        for g in self.g_loc.keys():
            self.g_points[g] = 200
        for e in self.e_loc.keys():
            self.e_points[e] = 200
        
        self.rangecells = defaultdict(list)
        for cell in self.opencells:
            x,y = cell
            rng = [(x-1,y),(x+1,y),(x,y-1),(x,y+1)]
            _ = [self.rangecells[cell].append(r) for r in rng if r in self.opencells]
    
    def targets(self, unit):
        targets = []
        if unit[0] == 'G':
            for e in self.e_loc.values():
                targets.extend(self.rangecells[e])
        elif unit[0] == 'E':
            for g in self.g_loc.values():
                targets.extend(self.rangecells[g])
        targets = [x for x in targets if not (x in self.g_loc.values() or x in self.e_loc.values())]
        return targets
    
    def nextstep(self, unit, routes = []):
        cellstovisit = self.freecells
        targets = self.targets(unit)
        routes = [[self.g_loc[unit]]] if not routes else routes
        solutions = []
        newroutes = []
        for route in sorted(routes):
            rangecells = [cell for cell in self.rangecells[route[-1]] if cell in cellstovisit]
            if rangecells:
                for rc in rangecells:
                    newroute = route.copy()
                    newroute.append(rc)
                    cellstovisit.remove(rc)
                    if rc in targets:
                        print('found shortest route to {}'.format(rc))
                        solutions.append(newroute)
                    else:
                        newroutes.append(newroute)
        if solutions:
            return [solution[1] for solution in sorted(solutions)][0]
        else:
            return self.nextstep(unit, newroutes)
    
    def _to_attack(self, coords, enemies): # enemies need to be sorted??
        rng = self.rangecells[coords]
        for key, value in enemies.items():
            if value in rng:
                return key
        return None
    
    def turn(self, unit):
        if unit[0] == 'G':
            coords = self.g_loc[unit]
            willattack = self._to_attack(coords, self.e_loc)
            if willattack:
                print('Unit {} will attack {}!'.format(unit, willattack))
            else:
                self.g_loc[unit] = self.nextstep(unit)
        elif unit[0] == 'E':
            coords = self.g_loc[unit]
            willattack = self._to_attack(coords, self.g_loc)
            if willattack:
                print('Unit {} will attack {}!'.format(unit, willattack))
            else:
                self.e_loc[unit] = self.nextstep(unit)
    
    @property
    def free(self):
        return len(self.freecells)
    
    @property
    def goblins(self):
        return len(self.g_loc)

    @property
    def elves(self):
        return len(self.e_loc)
    
    @property
    def freecells(self):
        return [e for e in self.opencells if not (e in self.g_loc.values() or e in self.e_loc.values())]


In [83]:
grid = Grid(t1.splitlines())

In [84]:
grid.turn('G3')

Unit G3 will attack E2!


In [71]:
grid.e_loc

{'E1': (2, 4), 'E2': (4, 5)}

In [78]:
r = grid.rangecells[grid.g_loc['G3']]
for key, value in grid.e_loc.items():
    print(value)
    if value in r:
        print(key)

(2, 4)
(4, 5)
E2


In [77]:
c = grid.g_loc['G3']
grid._to_attack(c, grid.e_loc)

In [50]:
grid.g_loc

{'G1': (1, 3), 'G2': (2, 5), 'G3': (3, 5), 'G4': (4, 3)}

In [10]:
def findshortestroute(grid, unit, routes = []):
    cellstovisit = grid.freecells
    targets = grid.targets(unit)
    routes = [[grid.g_loc[unit]]] if not routes else routes
    solutions = []
    newroutes = []
    for route in sorted(routes):
        rangecells = [cell for cell in grid.rangecells[route[-1]] if cell in cellstovisit]
        if rangecells:
            for rc in rangecells:
                newroute = route.copy()
                newroute.append(rc)
                cellstovisit.remove(rc)
                if rc in targets:
                    print('found shortest route to {}'.format(rc))
                    solutions.append(newroute)
                else:
                    newroutes.append(newroute)
    if solutions:
        return sorted(solutions)
    else:
        return findshortestroute(grid, unit, newroutes)

In [22]:
findshortestroute(grid, 'G1')

found shortest route to (2, 3)
found shortest route to (1, 4)


[[(1, 2), (1, 3), (1, 4)], [(1, 2), (1, 3), (2, 3)]]

In [14]:
grid.targets('G2')

[(1, 4), (2, 3), (5, 5)]

In [9]:
sorted(sol)

[[(1, 2), (1, 3), (1, 4)], [(1, 2), (2, 2), (2, 3)]]

## Backup code...

In [185]:
# the non-recursive option if needed:
cellstovisit = grid.freecells
targets = grid.targets('G1')
routes = [[grid.g_loc['G1']]]
solutions = []
while not solutions:
    newroutes = []
    for route in routes:
        rangecells = [cell for cell in grid.rangecells[route[-1]] if cell in cellstovisit]
        if rangecells:
            for rc in rangecells:
                newroute = route.copy()
                newroute.append(rc)
                cellstovisit.remove(rc)
                if rc in targets:
                    print('found shortest route to {}'.format(rc))
                    solutions.append(newroute)
                else:
                    newroutes.append(newroute)
    routes = newroutes
print('Solution(s) : {}'.format(solutions))

found shortest route to (2, 3)
found shortest route to (1, 4)
Solution(s) : [[(1, 2), (2, 2), (2, 3)], [(1, 2), (1, 3), (1, 4)]]


In [171]:
# flow:
if set(grid.rangecells[grid.e_loc['E1']]).intersection(set(grid.g_loc.values())):
    print('continue to attack')
else:
    targets = grid.targets('E1')

continue to attack
